In [64]:
import os
import nengo
import numpy as np
import tensorflow as tf
import keras
import nengo_dl
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense, BatchNormalization, Conv3D
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, ShuffleSplit


In [65]:
# Dataset path is by default saved in dataset_result/bci_dataset.npz
dataset_path = os.path.join('dataset_result', 'bci_dataset.npz')

# Load the numpy file containing the dataset
dataset = np.load(dataset_path)
features, labels = dataset['features'], dataset['labels'] # get features and labels

print('Features shape:', features.shape)
print('Labels shape:', labels.shape)

Features shape: (2976, 14, 36, 10)
Labels shape: (2976,)


In [66]:
# Set seed for consistency
seed = 4
np.random.seed(seed)
tf.random.set_seed(seed)

In [67]:
# Convert labels to one hot encoding
cat = OneHotEncoder()
labels = labels.reshape(-1, 1)
labels = cat.fit_transform(labels).toarray()
labels = labels.reshape((labels.shape[0], 1, -1))
print(labels)
print('Labels shape:', labels.shape)

# Reshape features for the NN
features = features.reshape((features.shape[0], 1, -1))
print('Features shape:', features.shape)

[[[0. 1.]]

 [[0. 1.]]

 [[0. 1.]]

 ...

 [[0. 1.]]

 [[0. 1.]]

 [[0. 1.]]]
Labels shape: (2976, 1, 2)
Features shape: (2976, 1, 5040)


In [68]:
def cnn_model_1():
    """
    Creates a CNN neural network
    :return: tensorflow model of the ANN
    """

    inp = Input(shape=(14, 360, 1), name='input_layer')
    conv2d = Conv2D(filters=9, kernel_size=(3, 3), activation=tf.nn.relu)(inp)
    dropout1 = Dropout(0.2, seed=seed)(conv2d)
    avg_pooling = AveragePooling2D(pool_size=(2, 2))(dropout1)
    flatten = Flatten()(avg_pooling)
    dense1 = Dense(1024, activation=tf.nn.relu)(flatten)
    batch_norm = BatchNormalization()(dense1)
    dense2 = Dense(512, activation=tf.nn.relu)(batch_norm)
    output = Dense(2, activation='softmax', name='output_layer')(dense2)

    return Model(inputs=inp, outputs=output)


def cnn_model_2():
    inp = Input(shape=(14, 360, 1), name='input_layer')
    conv1 = Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu)(inp)
    dropout1 = Dropout(0.2, seed=seed)(conv1)
    avg_pool1 = AveragePooling2D(pool_size=(2, 2))(dropout1)
    conv2 = Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu)(avg_pool1)
    dropout2 = Dropout(0.2)(conv2)
    avg_pool2 = AveragePooling2D(pool_size=(2, 2))(dropout2)
    flatten = Flatten()(avg_pool2)
    dense1 = Dense(512, activation=tf.nn.relu)(flatten)
    batch_norm = BatchNormalization()(dense1)
    dense2 = Dense(256, activation=tf.nn.relu)(batch_norm)
    output = Dense(2, activation=tf.nn.sigmoid, name='output_layer')(dense2)

    return Model(inputs=inp, outputs=output)


def cnn_model_3():
    inp = Input(shape=(14, 360, 1), name='input_layer')
    conv1 = Conv2D(filters=16, kernel_size=(5, 5), activation=tf.nn.relu, strides=(2, 2), padding='same')(inp)
    conv2 = Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu, strides=(2, 2), padding='same')(conv1)
    avg_pool1 = AveragePooling2D(pool_size=(2, 2), strides=2, padding='same')(conv2)
    dropout1 = Dropout(0.2)(avg_pool1)
    conv3 = Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu, padding='same')(dropout1)
    avg_pool2 = AveragePooling2D(pool_size=(2, 2), strides=2)(conv3)
    conv4 = Conv2D(filters=128, kernel_size=(3, 3), activation=tf.nn.relu, padding='same')(avg_pool2)
    avg_pool3 = AveragePooling2D(pool_size=(2, 2), strides=2, padding='same')(conv4)
    flatten = Flatten()(avg_pool3)
    dense1 = Dense(2048, activation=tf.nn.relu)(flatten)
    dropout2 = Dropout(0.2)(dense1)
    dense2 = Dense(1024, activation=tf.nn.relu)(dropout2)
    dropout3 = Dropout(0.2)(dense2)
    output = Dense(2, activation='sigmoid', name='output_layer')(dropout3)

    return Model(inputs=inp, outputs=output)


def dense_only_model():
    inp = Input(shape=(14, 360, 1), name='input_layer')
    flatten = Flatten()(inp)
    dense1 = Dense(1260, activation='relu')(flatten)
    dense2 = Dense(630, activation='relu')(dense1)
    dense3 = Dense(315, activation='relu')(dense2)
    dense4 = Dense(15, activation='relu')(dense3)
    output = Dense(2, activation='sigmoid', name='output_layer')(dense4)

    return Model(inputs=inp, outputs=output)

In [69]:
def run_ann(model, train, valid, test, params_save_path, iteration, shuffle_training=True, num_epochs=5):
    x_train, y_train = train[0], train[1]
    x_valid, y_valid = valid[0], valid[1]
    x_test, y_test = test[0], test[1]

    converter = nengo_dl.Converter(model)

    with nengo_dl.Simulator(converter.net, minibatch_size=16) as simulator:
        simulator.compile(optimizer=keras.optimizers.Adam(),
                          loss=keras.losses.BinaryCrossentropy(),
                          metrics=['accuracy'])

        input_layer = converter.inputs[model.get_layer('input_layer')]
        output_layer = converter.outputs[model.get_layer('output_layer')]

        simulator.fit(
            x={ input_layer: x_train }, y={ output_layer: y_train },
            validation_data=(
                { input_layer: x_valid }, { output_layer: y_valid }
            ),
            epochs=num_epochs,
            shuffle=shuffle_training,
            callbacks=[EarlyStopping(patience=8, verbose=1, restore_best_weights=True)] # early stop to avoid overfitting
        )

        simulator.save_params(params_save_path)
        ann_eval = simulator.evaluate(x={ input_layer: x_test }, y={ output_layer: y_test }) # evaluate accuracy
        print('{}. ann accuracy: {:5f}%'.format(iteration, ann_eval['probe_accuracy'] * 100)) # log accuracy
        return ann_eval['probe_accuracy'] # return accuracy


def run_snn(model, x_test, y_test, params_load_path, iteration, timesteps=50, scale_firing_rates=1000, synapse=0.01):
    converter = nengo_dl.Converter(
        model,
        swap_activations={ tf.nn.relu: nengo.SpikingRectifiedLinear() },
        scale_firing_rates=scale_firing_rates,
        synapse=synapse
    ) # create a Nengo converter object and swap all relu activations with spiking relu

    with converter.net:
        nengo_dl.configure_settings(stateful=False)

    input_layer = converter.inputs[model.get_layer('input_layer')] # input layer for simulator
    output_layer = converter.outputs[model.get_layer('output_layer')] # output layer for simulator

    x_test_tiled = np.tile(x_test, (1, timesteps, 1)) # tile test data to timesteps

    with nengo_dl.Simulator(converter.net) as simulator:
        simulator.load_params(params_load_path)

        predictions = simulator.predict({ input_layer: x_test_tiled })[output_layer] # get results from prediction
        predictions = predictions[:,-1,:] # get last time step

        predictions = np.argmax(predictions, axis=-1) # get argmax
        y_test = np.squeeze(y_test, axis=1) # remove time dimension from labels since its not relevant
        y_test = np.argmax(y_test, axis=-1) # get argmax of y test as well for comparison

        snn_acc = (predictions == y_test).mean()

    print('{}. SNN accuracy: {:5f}%'.format(
            iteration, snn_acc * 100
        ))

    return snn_acc

In [70]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=seed, shuffle=True)
print('x_train shape: {}, y_train shape: {}'.format(x_train.shape, y_train.shape))
print('x_test shape: {}, y_test shape: {}'.format(x_test.shape, y_test.shape))


# Create directory for ANN parameters used to construct the spiking network for Nengo
os.makedirs('nengo_params', exist_ok=True)
ann, snn = [], []

num_iterations = 1 # Number of cross validation iterations
iteration = 1 # Current iteration
shuffle_split = ShuffleSplit(n_splits=num_iterations, test_size=0.25, random_state=seed) # Shuffle split for cross valid

for train, valid in shuffle_split.split(x_train):
    print('Current iteration: ', iteration)
    x_train_curr, y_train_curr = x_train[train], y_train[train]
    x_valid_curr, y_valid_curr = x_train[valid], y_train[valid]

    params_path = os.path.join('nengo_params', 'params_{}'.format(iteration))
    model = cnn_model_2()

    # run ann
    ann_result = run_ann(model=model,
                         train=(x_train_curr, y_train_curr),
                         valid=(x_valid_curr, y_valid_curr),
                         test=(x_test, y_test),
                         params_save_path=params_path,
                         iteration=iteration,
                         num_epochs=80
                         )

    # run snn
    snn_result = run_snn(model=model,
                         x_test=x_test,
                         y_test=y_test,
                         params_load_path=params_path,
                         iteration=iteration)

    ann.append(ann_result)
    snn.append(snn_result)
    print(ann_result)
    print(snn_result)
    iteration += 1

x_train shape: (2232, 1, 5040), y_train shape: (2232, 1, 2)
x_test shape: (744, 1, 5040), y_test shape: (744, 1, 2)
Current iteration:  1
Build finished in 0:00:01                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/80
104/104 [==============================] - 45s 382ms/step - loss: 0.7802 - probe_loss: 0.7802 - probe_accuracy: 0.4941 - val_loss: 0.7968 - val_probe_loss: 0.7968 - val_probe_accuracy: 0.4393

c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\converter.py:326: UserWarning: Layer type Dropout does not have a registered converter. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\converter.py:326: UserWarning: Cannot convert BatchNormalization layer to native Nengo objects unless inference_only=True or layer.trainable=False. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\simulator.py:460: UserWarning: No GPU support detected. See https://www.nengo.ai/nengo-dl/installation.html#installing-tensorflow for instructions on setting up TensorFlow with GPU support.
  warnings.warn(
c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\simulator.py:1921: UserWarning: Number of elements in input data (558) is not evenly divisible by Simulator.minibatch_size (16); input data will be truncated.
  warnings.warn(
c:\dev\anaconda\envs\spiking-env\lib\site-packages\